### Some Notation:
```
#              -> Comment from RAPTOR's Original code or a general commented out statement
##             -> Own comment
##???? or ???? -> Needs attention
###D           -> Debug Purposes
```

In [1]:
from einsteinpy.symbolic import ChristoffelSymbols
from einsteinpy.symbolic.predefined import Kerr
from sympy import symbols
import numpy as np

# from einsteinpy.constant import c as sol

In [2]:
##INPUT PARAMETER
# // OBSERVER PARAMETERS
INCLINATION = 90
a = 0.998

##CONSTANTS
c = 1
r_s = 2

# // GLOBAL VARIABLES
# ////////////////////
DIM = 4

# // Metric
## Standard BL/KS coordinates; no logarithmic radius - ???? FOR NOW
logscale = False

IMG_WIDTH = 10 ## pixels | Reduced for perf reasons
IMG_HEIGHT = 10 ## pixels | Reduced for perf reasons
# IMG_WIDTH = 1000 # pixels | Original
# IMG_HEIGHT = 1000 # pixels | Original

# // Maximimum order of lensed image computed (0 = direct only)
max_order = 100
# max_order = 100 # | Original
# // CAMERA SWITCHES
maxsize = 100 ## | Reduced for perf reasons
#define maxsize 1000000 # | Original
# // Maximum number of integration steps
max_steps = 1e3 ## | Reduced for perf reasons
# max_steps = 1e5 # | Original

CAM_SIZE_X = 40 # Units of Rg | Rg = r_s/2 or GM/c^2
CAM_SIZE_Y = 40 # Units of Rg | Rg = r_s/2 or GM/c^2
STEPSIZE = 0.03

# // Camera distance from the sing.(units of Rg)
rcam = 1e4
# // Stop tracing at this distance from E.H. [BL]
cutoff_outer = (rcam * 1.01)

horizon_marg = (1e-1)
# // Cutoff outside or inside BH EH
cutoff_inner = (1. + np.sqrt(1. - a**2))*(1. + horizon_marg)

In [3]:
chris_sym = ChristoffelSymbols.from_metric(metric=Kerr())
# Evaluates connection coefficients at particular position
cs_lambda = chris_sym.tensor_lambdify()[1]

# Updates the vector y (containing position/velocity) by one RK4 step.
def rk4_step(y, dt):
    # Array containing all "update elements/# Coefficients" (4 times Nelements because RK4)
    coef = np.zeros((DIM * 2 * 4), dtype=float)

    # Create a copy of the "y vector" that can be shifted for the
    # separate function calls made by RK4
    yshift = np.copy(y)

    # fvector contains f(yshift), as applied to yshift (the 'current' y
    # during RK steps). It is used to compute the 'k-coefficients' (dx/##coef)
    fvector = np.zeros((DIM * 2), dtype=float)
    ##CONTAINS ALL 8 k-COEF (4 vel for pos and 4 acc for vel) NOT C-COEF, PER INT STEP

    # Compute the RK4 update coefficients ('K_n' in lit., 'dx/##coef' here)
    # int i, q;
    weights = np.array([0.5, 0.5, 1., 0.], dtype=float)

    for q in range(4):
        fvector = f_geodesic(yshift, fvector) # Apply function f to current y to obtain fvector
        for i in range(DIM * 2):
            coef[q * DIM * 2 + i] = dt * fvector[i] # Use fvector to fill coef
            yshift[i] = y[i] + coef[q * DIM * 2 + i] * weights[q] # Update y ##yshift

    # Update the y-vector (light ray)
    for i in range(DIM * 2):
        y[i] = y[i] + 1. / 6. * (coef[0 * DIM * 2 + i] +
                                    coef[1 * DIM * 2 + i] * 2. +
                                    coef[2 * DIM * 2 + i] * 2. +
                                    coef[3 * DIM * 2 + i])

    return y

In [4]:
# The function to be used by the integrator for GR geodesic calculations.
# y contains the 4-position and the 4-velocity for one lightray/particle.
## y is photon_u, for all intents and purposes
def f_geodesic(y, fvector): # real *y, real *fvector
    gamma_udd = np.zeros((4, 4, 4), dtype=float)
#     print('fvector: ', fvector) ###D

    # Initialize position, four-velocity, and four-acceleration vectors based
    # on values of y
    X_u = np.array([y[0], y[1], y[2], y[3]], dtype=float) # X
    U_u = np.array([y[4], y[5], y[6], y[7]], dtype=float) # dX/dLambda ## = U
    A_u = np.zeros(4, dtype=float) # d^2X/dLambda^2

    # Obtain the Christoffel symbols at the current location ##- X_u[i] ARE IN SPHERICAL POLAR
    gamma_udd = np.array(cs_lambda(t=X_u[0], r=X_u[1], theta=X_u[2], phi=X_u[3], a=a, c=c, r_s=r_s))
#     print('gamma_udd: ', gamma_udd) ###D

    # Compute 4-acceleration using the geodesic equation
    for i in range(DIM):
        for j in range(DIM):
            for k in range(DIM):
                A_u[i] -= gamma_udd[i, j, k] * U_u[j] * U_u[k]
    
#     print('fvector: ', fvector) ###D
    fvector[:4] = U_u[:]
    fvector[4:] = A_u[:]

    return fvector

In [5]:
# // Returns an appropriate stepsize dlambda, which depends on position & velocity
# // Ref. DOLENCE & MOSCIBRODZKA 2009
def stepsize(X_u, U_u): # real X_u[4], real U_u[4]
    SMALL =  1.e-20 ##PROTECTS AGAINST DIV-BY-0

    dlx1  = STEPSIZE / (np.abs(U_u[1]) + SMALL*SMALL)
    dlx2  = STEPSIZE * np.min([X_u[2], 1. - X_u[2]])/ (np.abs(U_u[2]) + SMALL*SMALL) ##SMALL FIXES WITH FUNCTION SWAPS
    dlx3  = STEPSIZE / (np.abs(U_u[3]) + SMALL*SMALL);

    idlx1 = 1. / (np.abs(dlx1) + SMALL*SMALL)
    idlx2 = 1. / (np.abs(dlx2) + SMALL*SMALL)
    idlx3 = 1. / (np.abs(dlx3) + SMALL*SMALL)

    return -np.max([1. / (idlx1 + idlx2 + idlx3), 1e-12])

In [6]:
# // Initialize a contravariant photon wave vector based on impact parameters
# // alpha and beta
# // Ref. Cunningham & Bardeen 1973
# // We want to pick E, L, Q based on impact params alpha, beta
# // Then construct k_u using E, L, Q
# // The photons all start at the camera location
def initialize_photon(alpha, beta, photon_u, t_init):
    mu0 = np.cos(INCLINATION / 180. * np.pi) ##INCLINATION IS 'i' IN DEGREES
    Xcam_u = np.array([t_init, rcam, np.arccos(mu0), 90./180. * np.pi])
    En = 1. ##ENERGY, E
    ll = -alpha * np.sqrt(1. - mu0**2)
    qq = beta**2 + mu0**2 * (alpha**2 - 1.)
    L1 = ll * En ##ANG MOM, L
    Q1 = qq * (En**2)
    k_d = np.zeros(4, dtype=float)
    k_u = np.zeros(4, dtype=float)
    sinc2 = np.sin(Xcam_u[2])**2
    cosc2 = np.cos(Xcam_u[2])**2

    # // Covariant wave vector entries are known:
    k_d[0] = -En
    k_d[3] = L1
    k_d[2] = np.sign(beta) * np.sqrt(np.abs(Q1 - L1 * L1 * (cosc2/sinc2) + (En**2)*cosc2))

    # // Construct contravariant wave vector k_u using the BL metric
    r       = rcam
    rfactor = 1.

    theta = np.arccos(mu0)
    sint  = np.sin(theta)
    cost  = np.cos(theta)
    sigma = r**2 + (a * cost)**2
    delta = r**2 + a**2 - 2. * r
    A_    = (r**2 + a**2) * (r**2 + a**2) - delta * a**2 * sint**2

    g_dd_11 = sigma / delta * rfactor**2
    g_uu_00 = -A_ / (sigma * delta)
    g_uu_03 = -2. * a * r / (sigma * delta)
    g_uu_33 = (delta - (a * sint)**2) / (sigma * delta * sint**2)
    g_uu_22 = 1. / sigma

    k_u[0] = g_uu_00 * k_d[0] + g_uu_03 * k_d[3]
    k_u[3] = g_uu_33 * k_d[3] + g_uu_03 * k_d[0]
    k_u[2] = g_uu_22 * k_d[2]
    k_u[1] = np.sqrt((-k_u[0]*k_d[0]-k_u[2]*k_d[2]-k_u[3]*k_d[3]) / g_dd_11)

    # // Place wave vector into "photon_u"
    photon_u[0] = Xcam_u[0]
    photon_u[1] = Xcam_u[1]
    photon_u[2] = Xcam_u[2]
    photon_u[3] = Xcam_u[3]
    photon_u[4] = k_u[0]
    photon_u[5] = k_u[1]
    photon_u[6] = k_u[2]
    photon_u[7] = k_u[3]
    
    return photon_u

In [7]:
##X_ext append -> IMPLEMENTED

##CAM_SIZE_X, CAM_SIZE_Y, IMAGE_WIDTH, IMAGE_HEIGHT, STEPSIZE -> model.in
##stopx values -> IGNORING FOR NOW
##INCLINATION -> USED 90 FIRST, FROM THE PAPER
## p -> IGNORING FOR NOW -- PURPOSE UNCLEAR -> LOOKS LIKE FOR USE IN GRMHD FOR SOME SORT OF STORAGE
## x, y -> core.c
## icur, Xcam, Ucam -> IGNORING FOR NOW
## intensityfield2, frequencies, t -> IGNORING FOR NOW

## intensityfield2 & frequencies should be removable IF NO RAD TRANS
## icur, Xcam, Ucam, t seem ununsed for RT
# // Integrate the null geodesic defined by "photon_u"
def integrate_geodesic(x, y):
    # (int icur,int x, int y, real intensityfield2[maxsize][num_indices],real *frequencies, real **** p,real t,real Xcam[4],real Ucam[4])
    # // Variables
    t_init = 0.
    dlambda_adaptive = 0.
    theta_turns = 0
    thetadot_prev = 0.
    X_u = np.zeros((4), dtype=float)
    k_u = np.zeros((4), dtype=float)
    alpha, beta = 0., 0.

    steps = 0

    photon_u = np.zeros(8, dtype=float)
    
    ##OWN LIST - STORES X_u in CARTESIAN
    X_ext = []

    # // Create initial ray conditions - LINEAR_IMPACT_CAM == True
    stepx = CAM_SIZE_X / IMG_WIDTH
    stepy = CAM_SIZE_Y / IMG_HEIGHT
    alpha = -CAM_SIZE_X * 0.5 + (x + 0.5) * stepx
    beta  = -CAM_SIZE_Y * 0.5 + (y + 0.5) * stepy

    photon_u = initialize_photon(alpha, beta, photon_u, t_init)

    # // Construct "photon_u"
    X_u[:] = photon_u[:4]
    
    ##OWN LIST APPEND - IN CARTESIAN
    theta_ext = np.radians(INCLINATION)
    x_ext = X_u[1] * np.sin(theta_ext) * np.cos(X_u[3])
    y_ext = X_u[1] * np.sin(theta_ext) * np.sin(X_u[3])
    z_ext = X_u[1] * np.cos(theta_ext)
    X_ext.append([x_ext, y_ext, z_ext]) # Initial X_u

    # // Current r-coordinate
    r_current = photon_u[1]

    # // Reset lambda and steps
    lambd = 0.
    steps = 0

    # Termination condition for ray
    TERMINATE = 0

    # // Trace light ray until it reaches the event horizon (##cutoff_inner) or the outer cutoff, or steps > max_steps
    # // Stop condition for BL coords
    while (r_current > cutoff_inner and r_current < cutoff_outer and steps < max_steps and not TERMINATE): # // && photon_u[0] < t_final){
        # // Current photon position/wave vector
        X_u[:] = photon_u[:4]
        k_u[:] = photon_u[4:]
    
        ##OWN LIST APPEND
        theta_ext = X_u[2]
        x_ext = X_u[1] * np.sin(theta_ext) * np.cos(X_u[3])
        y_ext = X_u[1] * np.sin(theta_ext) * np.sin(X_u[3])
        z_ext = X_u[1] * np.cos(np.radians(theta_ext))
        X_ext.append([x_ext, y_ext, z_ext])
        
        # // Possibly terminate ray to eliminate higher order images
        if thetadot_prev * photon_u[6] < 0. and steps > 2:
            theta_turns += 1
        thetadot_prev = photon_u[6]
        if ((beta < 0. and theta_turns > max_order) or (beta > 0. and theta_turns > (max_order + 1))):
            TERMINATE = 1

        # // Compute adaptive step size
        dlambda_adaptive = stepsize(X_u, k_u)

        # // Advance ray/particle
        photon_u = rk4_step(photon_u, dlambda_adaptive)

        X_u[:] = photon_u[:4]
        k_u[:] = photon_u[4:]

#         if X_u[1] > stopx[1] and  k_u[1] < 0: ##stopx values -> IGNORING FOR NOW
#             break

        r_current = photon_u[1]

        # // Update lambda, r_current, and steps.
        lambd += np.abs(dlambda_adaptive)
        r_current = photon_u[1]
        steps += 1
        
    return np.array(X_ext)


In [ ]:
##CALCULATION AND PLOTTING
from mpl_toolkits.mplot3d import Axes3D

import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

lmax = 1

for l in range(lmax):
    l1 = (int)(l*maxsize)
    l2 = (int)((l+1)*maxsize)

    if l2 > (IMG_WIDTH)*(IMG_HEIGHT):
        l2 = (IMG_WIDTH)*(IMG_HEIGHT)

    for i in range(l1, l2):
        y = (i / IMG_WIDTH)
        x = (i % IMG_WIDTH)
        print(i) ###D

        # // INTEGRATE THIS PIXEL'S GEODESIC ~~AND PERFORM RADIATIVE TRANSFER AT DESIRED FREQUENCIES, STORE RESULTS~~
        #     integrate_geodesic(icur,x,y,intensityfield2,frequencies,p,TIME_INIT,Xcam,Ucam);
        ##rays === X_ext -> Stores points in CARTESIAN, as x, y, z
        rays = integrate_geodesic(x, y) ##SHAPE = Nx3
        print(rays) ###D
        for xs, ys, zs in rays:
            ax.scatter(xs, ys, zs, color='black')

        ax.plot(rays[:][0], rays[:][1], rays[:][2], 'red')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')


plt.show()